# Feature Permutation

Theoretical background: https://christophm.github.io/interpretable-ml-book/feature-importance.html
API docs: https://captum.ai/api/feature_permutation.html
Output: Tensor of (#examples * #features * #perturbations_per_eval)

In [3]:
import torch
import scvi
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt

sc.set_figure_params(dpi=100, frameon=False, color_map='Reds', facecolor=None)
sc.logging.print_header()

Global seed set to 0


scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.23.3 scipy==1.9.1 pandas==1.4.4 scikit-learn==1.1.2 statsmodels==0.13.2 pynndescent==0.5.7


## load model

In [6]:
hlca_path = '../datasets/hlca.h5ad'
adata = sc.read(hlca_path)
adata

AnnData object with n_obs × n_vars = 584944 × 2000
    obs: 'sample', 'original_celltype_ann', 'study_long', 'study', 'last_author_PI', 'subject_ID', 'subject_ID_as_published', 'pre_or_postnatal', 'age_in_years', 'age_range', 'sex', 'ethnicity', 'mixed_ethnicity', 'smoking_status', 'smoking_history', 'BMI', 'known_lung_disease', 'condition', 'subject_type', 'cause_of_death', 'sample_type', 'anatomical_region_coarse', 'anatomical_region_detailed', 'tissue_dissociation_protocol', 'cells_or_nuclei', 'single_cell_platform', "3'_or_5'", 'enrichment', 'sequencing_platform', 'reference_genome_coarse', 'ensembl_release_reference_genome', 'cell_ranger_version', 'disease_status', 'fresh_or_frozen', 'cultured', 'cell_viability_%', 'comments', 'Processing_site', 'dataset', 'anatomical_region_level_1', 'anatomical_region_level_2', 'anatomical_region_level_3', 'anatomical_region_highest_res', 'age', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_ne

In [10]:
model = scvi.model.SCANVI.load('../models/scanvi_model/', adata)
model

INFO     File ../models/scanvi_model/model.pt already downloaded                                                   


/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/scvi/model/base/_utils.py:134: UserWarning: var_names for adata passed in does not match var_names of adata used to train the model. For valid results, the vars need to be the same and in the same order as the adata used to train the model.
  warnings.warn(
/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/scvi/data/fields/_layer_field.py:78: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(
/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/scvi/data/_utils.py:156: UserWarning: Category 18 in adata.obs['_scvi_labels'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/scvi/data/_utils.py:156: UserWarning: Category 18 in adata.obs['_scvi_labels'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(


ScanVI Model with the following params: 
unlabeled_category: unlabeled, n_hidden: 128, n_latent: 30, n_layers: 2, dropout_rate: 0.1, dispersion: gene, 
gene_likelihood: nb
Training status: Trained

## Applying feature permutation

### Measure against classification probabilities

In [12]:
from captum.attr import visualization as viz
from captum.attr import FeaturePermutation

Wrap `model.module.forward` because captum has an internal check that the inputs pass are tensors.

In [194]:
forw = lambda x, y, z: model.module.forward({'X': x, 'batch':y, 'labels':z}, compute_loss=False)[1]['px_scale']

In [195]:
model.module.forward(batch, compute_loss=False)[1]['px_scale'].shape

torch.Size([100, 2000])

Run captum.

In [196]:
ablator = FeatureAblation(forw)

In [197]:
%%time
# # per feature per output
# attribution_map = ablator.attribute(
#     (batch['X'], batch['batch'], batch['labels']),
# #     target=['Macrophages']*batch['X'].shape[1],
#     feature_mask=(torch.tensor(feature_mask), torch.tensor(feature_mask), torch.tensor(feature_mask)))

# per feature (aggregation) (takes longer)
attribution_map = ablator.attribute(
    (batch['X'], batch['batch'], batch['labels']),
    perturbations_per_eval=1,
#     show_progress=True
)

CPU times: user 1h 30min 49s, sys: 33min 49s, total: 2h 4min 38s
Wall time: 7min 50s


In [175]:
[i.shape for i in attribution_map]

[torch.Size([10000, 2000]), torch.Size([10000, 1]), torch.Size([10000, 1])]

In [177]:
[i.shape for i in attribution_map]

[torch.Size([10000, 2000]), torch.Size([10000, 1]), torch.Size([10000, 1])]

Captum returns an attribution map of either `tensor(n_features * output_size, n_features), n_inputs` or ?

In [166]:
attribution_map[0].shape

torch.Size([10000, 2000])

In [115]:
attribution_map

(tensor([[-1.7431e-08, -1.7431e-08, -1.7431e-08,  ..., -1.7431e-08,
          -1.7431e-08, -1.7431e-08],
         [-1.0250e-10, -1.0250e-10, -1.0250e-10,  ..., -1.0250e-10,
          -1.0250e-10, -1.0250e-10],
         [-1.9924e-08, -1.9924e-08, -1.9924e-08,  ..., -1.9924e-08,
          -1.9924e-08, -1.9924e-08],
         ...,
         [-8.1410e-07, -8.1410e-07, -8.1410e-07,  ..., -8.1410e-07,
          -8.1410e-07, -8.1410e-07],
         [-9.7044e-01, -9.7044e-01, -9.7044e-01,  ..., -9.7044e-01,
          -9.7044e-01, -9.7044e-01],
         [-1.0539e-06, -1.0539e-06, -1.0539e-06,  ..., -1.0539e-06,
          -1.0539e-06, -1.0539e-06]]),
 tensor([[ 1.2044e-12],
         [-4.3241e-15],
         [ 1.4188e-12],
         [ 1.7650e-12],
         [ 5.3920e-14],
         [ 1.0132e-12],
         [ 1.1324e-12],
         [ 2.9206e-12],
         [ 1.7511e-16],
         [ 1.1511e-12],
         [ 2.4874e-13],
         [ 1.7375e-12],
         [ 8.7122e-13],
         [ 0.0000e+00],
         [-2.1124e

In [116]:
attribution_map[0].shape

torch.Size([140, 2000])

In [117]:
attribution_map[1].shape

torch.Size([140, 1])

In [13]:
feature_perm = FeaturePermutation(model.module.classify)

In [22]:
ds_to_attribution = {}

for dataset in (adata.obs.study.cat.categories):
    adata_dataset_subset = adata[adata.obs.dataset == dataset]
    
    n_cells = 10
    scdl = model._make_data_loader(adata=adata_dataset_subset,
                                   indices=list(range(n_cells)),
                                   batch_size=100)
    batch = next(scdl.__iter__())

    attr = feature_perm.attribute((batch['X'], batch['batch']))
    ds_to_attribution[dataset] = attr

INFO     AnnData object appears to be a copy. Attempting to transfer setup.                                        


/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value
/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)
/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/scvi/data/fields/_layer_field.py:78: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(


INFO     AnnData object appears to be a copy. Attempting to transfer setup.                                        


/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value
/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)
/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/scvi/data/fields/_layer_field.py:78: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(


INFO     AnnData object appears to be a copy. Attempting to transfer setup.                                        


...

/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value
/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)
/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/scvi/data/fields/_layer_field.py:78: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(
/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/scvi/data/_utils.py:156: UserWarning: Category 18 in adata.obs['_scvi_labels'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
/h

INFO     AnnData object appears to be a copy. Attempting to transfer setup.                                        


/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value
/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)
/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/scvi/data/fields/_layer_field.py:78: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(


INFO     AnnData object appears to be a copy. Attempting to transfer setup.                                        


/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value
/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)
/home/lukas/anaconda3/envs/yuge_env/lib/python3.8/site-packages/scvi/data/fields/_layer_field.py:78: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(


In [23]:
ds_to_attribution

{'Banovich_Kropski_2020': (tensor([[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]),
  tensor([[0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
          [0.],
  

In [17]:
attr

(tensor([[ 0.0000e+00, -1.2140e-13,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00, -3.8333e-17,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00, -1.5093e-13,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00]]),
 tensor([[-8.9845e-11],
         [-2.7637e-13],
         [-1.2618e-10],
         [-1.2528e-10],
         [-8.5337e-10],
         [-9.2650e-11],
         [-1.0005e-10],
         [ 1.6167e-12],
         [-2.4662e-15],
         [-1.4053e-10],
         [-8.0917e-13],
         [-1.3370e-10],
         [-7.9163e-11],
         [ 1.1921e-07],
         [-3.4623e

In [29]:
# output shape of X
ds_to_attribution["Banovich_Kropski_2020"][0].shape

torch.Size([280, 2000])

In [30]:
# output shape of batch
ds_to_attribution["Banovich_Kropski_2020"][1].shape

torch.Size([280, 1])

## Collapsing into required output structure

The output that we desire is genes x cell types. Currently we have number_batches * (28 cell types * batch size) * number_genes and this 14 times for the 14 datasets.

In [32]:
# 1. Collapse datasets attributions into a single tuple of attributions
# 2. Collapse the batches and cell types into a single vector
# 3. Reshape into genes * cell types